In [3]:
import warnings
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [4]:
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error


In [5]:
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))


In [29]:
crypto = pd.read_csv('/Users/sabrinaa_le/Downloads/crypto-markets.csv')
btc = crypto.loc[crypto['name'] == 'Bitcoin']
btc = btc.drop(['slug', 'date', 'market', 'ranknow', 'volume', 'close_ratio', 'spread', 'symbol', 'name', 'open', 'high', 'low'], axis = 1)
btc

,close
0,134.21
1,144.54
2,139.00
3,116.99
4,105.21
5,97.75
6,112.50
7,115.91
8,112.30
9,111.50


In [ ]:
# evaluate parameters
p_values = rande(0, 5)
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(btc.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=34648038.617
ARIMA(0, 0, 1) MSE=9391142.955
ARIMA(0, 1, 0) MSE=162037.158
ARIMA(0, 1, 1) MSE=168137.500
ARIMA(0, 1, 2) MSE=174264.132
ARIMA(0, 2, 0) MSE=298375.249
ARIMA(0, 2, 1) MSE=183945.136
ARIMA(1, 0, 0) MSE=161731.632
ARIMA(1, 1, 0) MSE=176755.145
ARIMA(1, 2, 0) MSE=251558.609
ARIMA(2, 1, 0) MSE=181572.894
ARIMA(2, 2, 0) MSE=238843.536
ARIMA(4, 1, 0) MSE=187557.202
ARIMA(4, 2, 0) MSE=212611.235
ARIMA(4, 2, 1) MSE=192859.817
ARIMA(6, 1, 0) MSE=188584.400
ARIMA(6, 2, 0) MSE=219838.597
ARIMA(8, 1, 0) MSE=195801.168
ARIMA(8, 2, 0) MSE=211045.431


,close
0,134.21
1,144.54
2,139.00
3,116.99
4,105.21
5,97.75
6,112.50
7,115.91
8,112.30
9,111.50


In [19]:
import statsmodels.tsa.statespace
import statsmodels.api as sm
import math

In [28]:
#From R code, parameters are SARIMA(0, 1, 0)(0, 1, 1)[4]

data = btc
# define model configuration
my_order = (0, 1, 0)
my_seasonal_order = (0, 1, 1, 4)
# define model

closelog = np.log(btc['close'])
model = sm.tsa.SARIMAX(closelog, order=my_order, seasonal_order=my_seasonal_order)

model_fit = model.fit()
# one step forecast
yhat = model_fit.forecast()

In [22]:
btc[['close']]

,close
0,134.21
1,144.54
2,139.00
3,116.99
4,105.21
5,97.75
6,112.50
7,115.91
8,112.30
9,111.50
